In [1]:

# ========================== 📦 IMPORT SECTION ==========================
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import time
from ast import literal_eval
from typing import List, Dict

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

from ragas.dataset_schema import Sample, EvaluationDataset, SingleTurnSample
from src.tag.src.text2sql_pipeline import generate_sql  # path kamu
from src.tag.evaluation.run_text2sql import run_text2sql_workflow
#from src.tag.src.text2sqlchain2 import generate_sql
from src.tag.src.query_executor import execute_text2sql_response
from src.tag.evaluation.eval_metrics import evaluate_retriever  # gunakan retriever-style
from src.tag.database.db_connection import connect_db
from src.tag.database.schema_loader import load_schema

from langchain_community.utilities import SQLDatabase



d:\code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\code\src\tag\src\text2sqlchain_few.py:32: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="nomic-embed-text")


In [2]:
conn = connect_db()
schema = load_schema(conn)

In [3]:
DATASET_PATH = os.path.join("data", "Dataset Testing 2.xlsx")
df = pd.read_excel(DATASET_PATH)

dataset = []

# Filter for "is_valid" rows first, then sample
valid_df = df[df["is_valid"]]

# Sample 85 rows randomly from the valid_df
# If there are fewer than 85 valid rows, it will take all of them.
sampled_df = valid_df.sample(n=min(85, len(valid_df)), random_state=42) # random_state for reproducibility

for i, row in sampled_df.iterrows():
    dataset.append({
        "user_input": str(row["user_input"]),
        "reference_contexts": literal_eval(row["reference_contexts_2"])
    })

print(f"Jumlah soal: {len(dataset)}")

Jumlah soal: 85


In [ ]:
DATASET_PATH = os.path.join("data", "Dataset Testing 2.xlsx")
df = pd.read_excel(DATASET_PATH)

dataset = []

sampled_df = valid_df.sample(n=min(85, len(valid_df)), random_state=42) # random_state for reproducibility

for i, row in df.iterrows():
    if row["is_valid"]:
        dataset.append({
            "user_input": str(row["user_input"]),
            "reference_contexts": literal_eval(row["reference_contexts_2"])
        })
    if len(dataset) == 85:
        break
print(f"Jumlah soal: {len(dataset)}")


Jumlah soal: 85


In [4]:
def retrieve_contexts_from_text2sql_zero(question: str) -> list[str]:
    try:
        response = generate_sql(schema, question, top_k=100, shot_mode="zero-shot", llm_mode="gemini")
        rows, columns = execute_text2sql_response(conn, response)
        if not rows:
            return ["data tidak ditemukan"]
        return [" | ".join(map(str, row)) for row in rows]
    except Exception as e:
        print(f"[!] Error: {question} → {e}")
        return []


In [4]:
def retrieve_contexts_from_text2sql_few(question: str) -> list[str]:
    try:
        response = generate_sql(schema, question, top_k=100, shot_mode="few-shot", llm_mode="gemini")
        rows, columns = execute_text2sql_response(conn, response)
        if not rows:
            return ["data tidak ditemukan"]
        return [" | ".join(map(str, row)) for row in rows]
    except Exception as e:
        print(f"[!] Error: {question} → {e}")
        return []


In [5]:
samples = []

for item in tqdm(dataset, desc="Menjalankan Text2SQL dan Eksekusi"):
    q = item["user_input"]
    ref = item["reference_contexts"]
    ret = retrieve_contexts_from_text2sql_zero(q)

    sample = SingleTurnSample(
        question=q,
        reference_contexts=ref,
        retrieved_contexts=ret
    )
    samples.append(sample)

evaluation_dataset = EvaluationDataset(samples)


Menjalankan Text2SQL dan Eksekusi:   0%|          | 0/15 [00:00<?, ?it/s]d:\code\src\tag\src\text2sql_pipeline.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
d:\code\src\tag\src\text2sql_pipeline.py:26: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run(inputs).strip()
Menjalankan Text2SQL dan Eksekusi: 100%|██████████| 15/15 [00:32<00:00,  2.17s/it]


In [6]:
result = evaluate_retriever(evaluation_dataset, experiment_name="tag_retriever_v1")
df_result_zero = result.to_pandas()
df_result_zero


Evaluating: 100%|██████████| 30/30 [00:01<00:00, 19.51it/s]


,retrieved_contexts,reference_contexts,precision,recall
0,[data tidak ditemukan],[(1) Persentase TKDN untuk belanja modal (cape...,0.000000,0.0
1,[data tidak ditemukan],[Lembaga Penyiaran Asing dilarang didirikan di...,0.000000,0.0
2,[202403006500400 | 4 | Seleksi pengguna Pita F...,[(1) Pelaksanaan Diklat REOR sebagaimana dimak...,0.000000,0.0
3,[Jaringan Dokumentasi dan Informasi Hukum Keme...,[Dalam Peraturan Menteri ini yang dimaksud den...,0.000000,0.0
4,[22 | (1) Registrar Nama Domain Selain Instans...,[Registri Nama Domain dan Registrar Nama Domai...,0.000000,0.0
5,[data tidak ditemukan],[Informasi tarif retail layanan jelajah (roami...,0.000000,0.0
6,[201503017500100 | 1 | Pembaca kartu cerdas ni...,[Pembaca kartu cerdas nirkontak (Contactless S...,1.000000,1.0
7,[6 | (1) Laporan sebagaimana dimaksud dalam Pa...,[(1) Laporan sebagaimana dimaksud dalam Pasal ...,0.000000,0.0
8,[data tidak ditemukan],[(1) Permohonan Nomor PI dapat dilakukan oleh ...,0.000000,0.0
9,[data tidak ditemukan],[Setiap alat dan perangkat telekomunikasi jara...,0.000000,0.0


In [7]:
avg_precision = df_result_zero['precision'].mean()
avg_recall = df_result_zero['recall'].mean()

print(f"Rata-rata Precision: {avg_precision:.4f}")
print(f"Rata-rata Recall: {avg_recall:.4f}")

Rata-rata Precision: 0.0793
Rata-rata Recall: 0.1667


In [5]:
import time
from tqdm import tqdm

samples = []

for i, item in enumerate(tqdm(dataset, desc="Menjalankan Text2SQL dan Eksekusi")):
    q = item["user_input"]
    ref = item["reference_contexts"]

    try:
        ret = retrieve_contexts_from_text2sql_few(q)
    except Exception as e:
        print(f"[!] Error executing SQL for question: {q}")
        print(f"    → {e}")
        
        # Lakukan rollback untuk mengakhiri transaction yang gagal
        try:
            conn.rollback()
            print("[✓] Transaction rollback executed.")
        except Exception as rollback_err:
            print(f"[X] Failed to rollback transaction: {rollback_err}")
        
        ret = []  # Tetap buat list kosong agar tidak error saat membuat sample

    sample = SingleTurnSample(
        question=q,
        reference_contexts=ref,
        retrieved_contexts=ret
    )
    samples.append(sample)

    # Pause setiap 10 pertanyaan
    if (i + 1) % 10 == 0:
        print(f"[i] Processed {i+1} samples. Sleeping for 15 seconds...")
        time.sleep(15)

evaluation_dataset_few = EvaluationDataset(samples)


Menjalankan Text2SQL dan Eksekusi:   0%|          | 0/85 [00:00<?, ?it/s]d:\code\src\tag\src\text2sql_pipeline.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
d:\code\src\tag\src\text2sql_pipeline.py:30: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run(inputs).strip()
Menjalankan Text2SQL dan Eksekusi:  11%|█         | 9/85 [00:41<05:51,  4.62s/it]

[i] Processed 10 samples. Sleeping for 15 seconds...


Menjalankan Text2SQL dan Eksekusi:  22%|██▏       | 19/85 [01:40<04:56,  4.50s/it]

[i] Processed 20 samples. Sleeping for 15 seconds...


Menjalankan Text2SQL dan Eksekusi:  34%|███▍      | 29/85 [02:39<04:26,  4.75s/it]

...

Menjalankan Text2SQL dan Eksekusi:  81%|████████  | 69/85 [06:41<01:19,  4.95s/it]

[i] Processed 70 samples. Sleeping for 15 seconds...


Menjalankan Text2SQL dan Eksekusi:  93%|█████████▎| 79/85 [07:37<00:25,  4.28s/it]

[i] Processed 80 samples. Sleeping for 15 seconds...


Menjalankan Text2SQL dan Eksekusi: 100%|██████████| 85/85 [08:19<00:00,  5.88s/it]


In [6]:
result = evaluate_retriever(evaluation_dataset_few, experiment_name="tag_retriever_v2")
df_result_few = result.to_pandas()
df_result_few


Evaluating:   6%|▋         | 11/170 [00:00<00:01, 107.76it/s]Exception raised in Job[58]: ValidationError(1 validation error for SingleTurnSample
reference
  Input should be a valid string [type=string_type, input_value=38, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type)
Exception raised in Job[59]: ValidationError(1 validation error for SingleTurnSample
reference
  Input should be a valid string [type=string_type, input_value=38, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type)
Evaluating: 100%|██████████| 170/170 [00:05<00:00, 32.22it/s]


,retrieved_contexts,reference_contexts,precision,recall
0,[diubah_oleh | Peraturan Menteri Komunikasi da...,[{'title': 'Peraturan Menteri Komunikasi dan I...,0.865385,0.150442
1,[data tidak ditemukan],[(1) Setiap Orang dengan sengaja dan tanpa hak...,0.000000,0.000000
2,[data tidak ditemukan],"[{'number': '1'}, {'number': '3'}, {'number': ...",0.000000,0.000000
3,[PSE Lingkup Privat User Generated Content | P...,[Dalam Peraturan Menteri ini yang dimaksud den...,0.000000,0.000000
4,[1 | Dalam Peraturan Menteri ini yang dimaksud...,"[(1) LPP, LPS, dan LPK jasa Penyiaran televisi...",0.153846,1.000000
...,...,...,...,...
80,[a. bahwa ketentuan Pasal 20 Undang-Undang Nom...,[{'consideration': 'a. bahwa ketentuan Pasal 2...,1.000000,1.000000
81,[data tidak ditemukan],[{'node': 'Pasal 26 UU Nomor 19 Tahun 2016 ten...,0.000000,0.000000
82,[data tidak ditemukan],"[(1) Pengelola Warnet, Hotspot dan sejenisnya ...",0.000000,0.000000
83,[90 | Peran Pemerintah dalam penyelenggaraan s...,[(1) Penyelenggara Agen Elektronik wajib:\n\ta...,0.000000,0.000000


In [7]:
from IPython.display import display

zero_precision_recall_few = df_result_few[
	(df_result_few['precision'] == 0.0) &
	(df_result_few['recall'] == 0.0) &
	(df_result_few['retrieved_contexts'].apply(lambda x: x != ['data tidak ditemukan']))
]
count_zero_precision_recall_few = len(zero_precision_recall_few)
#print(zero_precision_recall_few)


display(zero_precision_recall_few)
print(f"Jumlah kasus precision=0 & recall=0 (bukan 'data tidak ditemukan'): {count_zero_precision_recall_few}")

zero_precision_recall_few.to_csv("salahjawab_v1.csv", index=False)
print("Hasil evaluasi telah disimpan ke 'salahjawab_v1.csv'.")


,retrieved_contexts,reference_contexts,precision,recall
3,[PSE Lingkup Privat User Generated Content | P...,[Dalam Peraturan Menteri ini yang dimaksud den...,0.0,0.0
5,[Internet Exchange Point | Internet Exchange P...,[Dalam Peraturan Menteri ini yang dimaksud den...,0.0,0.0
10,[Tanda Tangan Elektronik | Tanda Tangan Elektr...,[Dalam Peraturan Menteri ini yang dimaksud den...,0.0,0.0
13,[1 | Dalam Peraturan Menteri ini yang dimaksud...,[(1) Pelaksana penyedia berhak mendapatkan aks...,0.0,0.0
14,[2008-04-21],[{'title': 'Pasal 40 UU Nomor 1 Tahun 2024 ten...,0.0,0.0
25,[26 | (1) Lembaga Pelatihan Terakreditasi berh...,[(1) Lembaga Pelatihan Terakreditasi berhak me...,0.0,0.0
32,[9 | Pasal 9 PERMENKOMINFO Nomor 11 Tahun 2018...,[{'title': 'Pasal 59 PERMENKOMINFO Nomor 7 Tah...,0.0,0.0
36,"[Pasal 39 PP Nomor 52 Tahun 2000, Pasal 45 PP ...",[{'title': 'Pasal 6 PERMENKOMINFO Nomor 1/PER/...,0.0,0.0
40,[43 | (1) Selain Penyidik Pejabat Polisi Negar...,[(1) Informasi Elektronik dan/atau Dokumen Ele...,0.0,0.0
42,[6 | (1) Laporan sebagaimana dimaksud dalam Pa...,[(1) Laporan sebagaimana dimaksud dalam Pasal ...,0.0,0.0


Jumlah kasus precision=0 & recall=0 (bukan 'data tidak ditemukan'): 22
Hasil evaluasi telah disimpan ke 'salahjawab_v1.csv'.


In [11]:
rows_data_tidak_ditemukan = df_result_few[df_result_few['retrieved_contexts'].apply(lambda x: x == ['data tidak ditemukan'])]
count_rows_data_tidak_ditemukan = len(rows_data_tidak_ditemukan)
print(count_rows_data_tidak_ditemukan)

28


In [40]:
df_result_few.to_csv("tag_retriever_v2.csv", index=False)
print("Hasil evaluasi telah disimpan ke 'tag_retriever_v2.csv'.")

Hasil evaluasi telah disimpan ke 'tag_retriever_v2.csv'.


In [9]:
avg_precision = df_result_few['precision'].mean()
avg_recall = df_result_few['recall'].mean()

print(f"Rata-rata Precision: {avg_precision:.4f}")
print(f"Rata-rata Recall: {avg_recall:.4f}")

Rata-rata Precision: 0.3251
Rata-rata Recall: 0.3552


In [ ]:
df_result.to_json("tag_retriever_v1.json", index=False)
print("Hasil evaluasi telah disimpan ke 'tag_retriever_v1.json'.")